In [19]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error 

import pandas as pd
import numpy as np

In [20]:
# Gather Data

boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)

log_prices = np.log(boston_dataset.target)
# log_prices.shape
target = pd.DataFrame(log_prices, columns=['PRICE'])
target.shape

(506, 1)

In [21]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

# property_stats = np.ndarray(shape=(1, 11))
# property_stats[0][CRIME_IDX] = features['CRIM'].mean()
# property_stats[0][ZN_IDX] = features['ZN'].mean()
# property_stats[0][CHAS_IDX] = features['CHAS'].mean()

property_stats = features.mean().values.reshape(1, 11)
property_stats

array([[3.59376071e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [22]:
type(features.mean())
type(features.mean().values)
features.mean().values.shape
features.mean().values.reshape(1, 11)

array([[3.59376071e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [23]:
features.mean().values


array([3.59376071e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
       6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
       1.84555336e+01, 3.56674032e+02, 1.26530632e+01])

In [24]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)
MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [25]:
def get_log_estimate(nr_rooms, students_per_classroom, next_to_river=False, high_confidence=True):
    
    # Configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
        
    # Make Prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calc Range
    
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
        
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    
    return log_estimate, upper_bound, lower_bound, interval

In [26]:
get_log_estimate(3, 20, next_to_river=True, high_confidence=False)

(2.7784264893000006, 2.965972198228978, 2.590880780371023, 68)

In [27]:
np.median(boston_dataset.target)

21.2

In [28]:
# convert log price estimate using 1970s prices, upper and lower bounds to todays prices,
# rounding values to nearest 1000 dollars

zillow_median_price = 583.3
scale_factor = zillow_median_price / np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_estimate(9, 15, False, False)

# Convert To Todays Dollars

dollar_est = np.e**log_est * 1000 * scale_factor
dollar_high = np.e**upper * 1000 * scale_factor
dollar_low = np.e**lower * 1000 * scale_factor

# Round the dollar values to nearest thousand

rounded_est = np.around(dollar_est, -3)
rounded_high = np.around(dollar_high, -3)
rounded_low = np.around(dollar_low, -3)

print(f'The Estimated property value is {rounded_est}.')
print(f'At {conf}% confidence the valuation range is USD {rounded_low} at the lower and {rounded_high} at the high end.')

The Estimated property value is 826000.0.
At 68% confidence the valuation range is USD 685000.0 at the lower and 996000.0 at the high end.


In [29]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """Estimate the price of a property in Boston.
    Keyword arguments:
    rm -- number of rooms in the property
    ptratio -- number of students per teacher in the classroom for the school in the area
    chas -- True if the property is next to the river, False otherwise
    large_range -- True for a 95% prediction interval, False for a 68% interval.
    
    """
    
    if rm < 1 or ptratio < 1:
        print('That is unrealistic.Try again')
        return

    log_est, upper, lower, conf = get_log_estimate(rm, ptratio, chas, large_range)

    # Convert To Todays Dollars

    dollar_est = np.e**log_est * 1000 * scale_factor
    dollar_high = np.e**upper * 1000 * scale_factor
    dollar_low = np.e**lower * 1000 * scale_factor

    # Round the dollar values to nearest thousand

    rounded_est = np.around(dollar_est, -3)
    rounded_high = np.around(dollar_high, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'The Estimated property value is {rounded_est}.')
    print(f'At {conf}% confidence the valuation range is USD {rounded_low} at the lower and {rounded_high} at the high end.')

In [30]:
get_dollar_estimate(5, -1, True)

That is unrealistic.Try again


In [31]:
import boston_valuation as val

val.get_dollar_estimate(6, 12, True)

The Estimated property value is 783000.0.
At 95% confidence the valuation range is USD 538000.0 at the lower and 1140000.0 at     the high end.
